In [1]:
import sys

import sklearn
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

In [2]:
train = pd.read_csv('data/house-prices/train.csv', index_col=['Id'])
test = pd.read_csv('data/house-prices/test.csv', index_col=['Id'])

In [3]:
train_no_sale_price = train.drop(labels=['SalePrice'], axis=1)
train_sale_price = train['SalePrice']

x_train, x_test, y_train, y_test = train_test_split(train_no_sale_price, 
                                                    train_sale_price, 
                                                    test_size=0.3, 
                                                    random_state=4330)

In [4]:
regressor = LinearRegression()

In [5]:
numeric_column_names = x_train.describe().columns
categorical_columns = train.dtypes[train.dtypes == 'object'].index

We will fill the missing values with the mean value of a given column

In [6]:
for index in numeric_column_names:
    x_train[index] = x_train[index].fillna(x_train.describe()[2:3].mean()[index])
    x_test[index] = x_test[index].fillna(x_train.describe()[2:3].mean()[index])

c:\users\sr\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\sr\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
train = train.fillna("")
encoders = {col: LabelEncoder().fit(train[col]) for col in categorical_columns}

def encode_categorical(data, columns, encoders):
    data = data.fillna("")
    return pd.DataFrame({col: encoders[col].transform(data[col]) for col in columns},
                        index = data.index)

train_encoded = encode_categorical(train, categorical_columns, encoders)

one_hot_encoder = OneHotEncoder().fit(train_encoded)

c:\users\sr\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


After trial and error droping the tables : 'LotFrontage','BsmtFinSF1','TotalBsmtSF' ,'GarageYrBlt','OpenPorchSF','3SsnPorch', 'MiscVal','MoSold','LandContour', 'Utilities','LandSlope','Exterior1st','Exterior2nd','MasVnrType', 'Electrical' ,'FireplaceQu' ,'GarageType', 'GarageFinish' ,'Fence' 
Seemed to increase the train and test score of the model

In [55]:
numeric_column_names_drop = numeric_column_names.drop(['LotFrontage','BsmtFinSF1','TotalBsmtSF' ,
                                'GarageYrBlt','OpenPorchSF','3SsnPorch',
                                 'MiscVal','MoSold' ])

drop_train_X = x_train[numeric_column_names_drop]


drop_test_X = x_test[numeric_column_names_drop]

categorical_columns_drop = categorical_columns
categorical_columns_drop = categorical_columns_drop.drop(['LandContour', 'Utilities',
                            'LandSlope','Exterior1st','Exterior2nd','MasVnrType'
                            ,'Electrical','FireplaceQu','GarageType','GarageFinish'
                            ,'Fence'])

drop_cat_train_X = x_train[categorical_columns_drop]

drop_cat_test_X  = x_test[categorical_columns_drop]


train_encoded = encode_categorical(train, categorical_columns_drop, encoders)
one_hot_encoder = OneHotEncoder().fit(train_encoded)


train_encoded2 = encode_categorical(train, categorical_columns_drop, encoders)
one_hot_encoder2 = OneHotEncoder().fit(train_encoded2)

one_hot_drop_x_train = one_hot_encoder.transform(encode_categorical(drop_cat_train_X, categorical_columns_drop, encoders))
one_hot_drop_x_test  = one_hot_encoder.transform(encode_categorical(drop_cat_test_X, categorical_columns_drop, encoders))


new_drop_x_train = pd.np.concatenate([one_hot_drop_x_train.todense(), drop_train_X], axis=1)
new_drop_x_test = pd.np.concatenate([one_hot_drop_x_test.todense(), drop_test_X], axis=1)

c:\users\sr\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
c:\users\sr\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this wa

After testing different settings for Ridge the deffault options seemed to give the default resoult along with Alpha set to 9

In [56]:
from sklearn.linear_model import Ridge


alphas  = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17, 18, 19 , 20, 21 , 22 , 23, 24]

for alpha in alphas:
    all_data_lr = Ridge(alpha = alpha,solver='auto' ).fit(new_drop_x_train, pd.np.log10(y_train))
    print('alpha:', alpha)
    print(all_data_lr.score(new_drop_x_train, pd.np.log10(y_train)), all_data_lr.score(new_drop_x_test, pd.np.log10(y_test)))
    print()

alpha: 1
0.9247926235918801 0.9330018457555824

alpha: 2
0.9197894141513961 0.9341793519350333

alpha: 3
0.916477073717634 0.9347152773238976

alpha: 4
0.9140054440069947 0.9350122995599409

alpha: 5
0.9120307997329244 0.9351879175494275

alpha: 6
0.9103832052882006 0.9352924660948134

alpha: 7
0.9089667912245137 0.9353516407862102

alpha: 8
0.9077223376852547 0.935380029804901

alpha: 9
0.9066107684862366 0.9353865550541068

alpha: 10
0.9056049725898748 0.9353769893767953

alpha: 11
0.9046853833762198 0.9353552387901081

alpha: 12
0.9038374220734076 0.9353240444173481

alpha: 13
0.9030499371734642 0.935285389264397

alpha: 14
0.9023142087359505 0.9352407452249485

alpha: 15
0.9016232892187863 0.9351912290813162

alpha: 16
0.9009715532243344 0.9351377044030716

alpha: 17
0.9003543815665881 0.9350808500796839

alpha: 18
0.8997679343484356 0.935021207607919

alpha: 19
0.8992089845962742 0.9349592144662168

alpha: 20
0.8986747940594986 0.9348952281476806

alpha: 21
0.8981630189776846 0.93

In [43]:
train = pd.read_csv('data/house-prices/train.csv', index_col=['Id'])
test = pd.read_csv('data/house-prices/test.csv', index_col=['Id'])

def numer_fillna(data):
    _numeric= data[numeric_column_names_drop]
    for index in numeric_column_names_drop:
        _numeric[index] = data[index].fillna(data.describe()[1:2].mean()[index])
    return _numeric

def pipeline_data(data):
    _numeric = numer_fillna(data)
    _categorical = data[categorical_columns_drop].fillna("")
    _encoded = encode_categorical(_categorical, categorical_columns_drop, encoders)
    _one_hot_encoded = one_hot_encoder2.transform(_encoded)
    _merged = pd.np.concatenate([_one_hot_encoded.todense(), _numeric], axis=1)
    return _merged

In [44]:
full_x = pipeline_data(train)
full_y = train['SalePrice']

# Fix test set missing data problems
test['MSZoning'].fillna('RL', inplace=True)
test['KitchenQual'].fillna('TA', inplace=True)
test['Functional'].fillna('Typ', inplace=True)
test['SaleType'].fillna('WD', inplace=True)

test_set = pipeline_data(test)

c:\users\sr\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
c:\users\sr\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [19]:
ridge = Ridge(alpha = 9 , solver='auto' ).fit(full_x, pd.np.log10(full_y))

In [20]:
predictions = 10**ridge.predict(test_set)
predictions_df = pd.DataFrame({"Id":test.index, "SalePrice":predictions})
predictions_df.to_csv("Resoults/Ridge.csv", index=False)
predictions_df.head()

,Id,SalePrice
0,1461,113890.787904
1,1462,150023.741880
2,1463,169633.212103
3,1464,191785.266429
4,1465,202666.670234


This gets a score of 0.12578 Rank 1437 (durring the time of writing this). I somehow made some chang (not sure what) and was able to get e slightly better score of 0.12549.

Next we will try to get a better score with Lasso

In [54]:
from sklearn import linear_model
for alpha in [0.01, 1 ,2 ,5 , 8 , 10 , 15 , 20 ,30 ,45 ,70 , 100 , 150 , 180 ,190, 200 ,210 ,220,250 , 270, 300]:
    lassoModel = linear_model.Lasso(alpha = alpha , max_iter=2000,
                                   warm_start=False, positive=False, 
                                random_state=None, selection='random').fit(new_drop_x_train, y_train)
    lasso_trainS = lassoModel.score(new_drop_x_train, y_train)
    lasso_testS  = lassoModel.score(new_drop_x_test, y_test)
    print('alpha:', alpha)
    print(lasso_trainS)
    print(lasso_testS)
    print()

c:\users\sr\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


alpha: 0.01
0.929574352228526
0.8796968384629401



c:\users\sr\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


alpha: 1
0.9295510241155998
0.880419412335854



c:\users\sr\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


...

c:\users\sr\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


alpha: 5
0.9291560282218005
0.881600867280406

alpha: 8
0.9287316617373305
0.8822946610546228

alpha: 10
0.928515318829886
0.8831669972167888

alpha: 15
0.927927357494538
0.8850732015229075

alpha: 20
0.9272662517492297
0.8866106771607285



c:\users\sr\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


alpha: 30
0.9257302087703443
0.8892956468485036

alpha: 45
0.9232239305877662
0.8933309441664251

alpha: 70
0.9190446595038442
0.8975355671529872

alpha: 100
0.9142299867911328
0.9001001507659121

alpha: 150
0.9050459269580482
0.9026790668654786

alpha: 180
0.8985496800258138
0.9026783653709098

alpha: 190
0.8962150896204558
0.9024528006945796

alpha: 200
0.8937844545109823
0.9021314568172577

alpha: 210
0.8912577604047475
0.9017241218214309

alpha: 220
0.8888318247425637
0.9010980644357877

alpha: 250
0.8820343590540082
0.8983678323864173

alpha: 270
0.8770275250094277
0.8963187868390724

alpha: 300
0.8699915242294037
0.89352758943129



c:\users\sr\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [48]:
lasso = linear_model.Lasso(alpha = 190 , max_iter=2000,
              warm_start=False, positive=False, 
              random_state=None, selection='random').fit(full_x, pd.np.log10(full_y))

In [49]:
predictions2 = 10**lasso.predict(test_set)
predictions_df2 = pd.DataFrame({"Id":test.index, "SalePrice":predictions2})
predictions_df2.to_csv("Resoults/Lasso.csv", index=False)
predictions_df2.head()

,Id,SalePrice
0,1461,167807.945101
1,1462,170448.845778
2,1463,170009.674333
3,1464,166187.153457
4,1465,161379.036456


This had a really bad score in kaggle but i think it might be becouse im not exporting correctly but i can't seem to find the where the issue is here :S